In [1]:
import os
from keras import Model, Input
from keras.optimizers import Adam
from keras.applications.mobilenet import MobileNet, preprocess_input 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from livelossplot.inputs.keras import PlotLossesCallback
from pathlib import Path
import numpy as n

2025-01-19 14:45:59.288379: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-01-19 14:45:59.380179: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-01-19 14:45:59.846003: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2025-01-19 14:45:59.846047: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinf

## Analise dos dados

In [2]:
path = "../../data/treino"
pathTest = "../../data/teste"


def listdir(dir):
    filenames = os.listdir(dir)
    
    total = 0
    for files in filenames:
        count = 0
        file_path = os.path.join(dir, files)
        for file in os.listdir(file_path):
            count += 1
        print(files + ": " + str(count))
        total += count

    print(total)
    
print("Para Treino")
listdir(path)
print("Para Teste")
listdir(pathTest)

Para Treino
P-70: 110
P17: 171
P-21: 137
P-13: 52
P-1: 230
P-19: 192
P-62: 216
P-08: 57
P-3: 78
P-09: 108
P-22: 283
P-04: 42
P-16: 156
P-69_: 62
P-01: 109
P-11: 263
2266
Para Teste
P-70: 36
P17: 54
P-21: 46
P-13: 17
P-1: 77
P-19: 64
P-62: 72
P-08: 19
P-3: 26
P-09: 36
P-22: 94
P-04: 14
P-16: 51
P-69_: 20
P-01: 36
P-11: 87
749


In [3]:

BATCH_SIZE = 64

train_generator = ImageDataGenerator(
                                    rotation_range=90,
                                    brightness_range=[0.1, 0.7],
                                    width_shift_range=0.5,
                                    height_shift_range=0.5,
                                    horizontal_flip=True,
                                    vertical_flip=True,
                                    validation_split=0.15,
                                    preprocessing_function=preprocess_input) # VGG16 preprocessing

test_generator = ImageDataGenerator(preprocessing_function=preprocess_input) # VGG16 preprocessing

In [4]:
class_subset = sorted(os.listdir('../../data/imagens'))
training =  train_generator.flow_from_directory(directory='../../data/treino',
                                               target_size=(224, 224),
                                               class_mode='categorical',
                                               classes=class_subset,
                                               subset='training',
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               seed=42)

validation = train_generator.flow_from_directory('../../data/treino',
                                               target_size=(224, 224),
                                               class_mode='categorical',
                                               classes=class_subset,
                                               subset='validation',
                                               batch_size=BATCH_SIZE,
                                               shuffle=True,
                                               seed=42)

testing = test_generator.flow_from_directory('../../data/teste',
                                             target_size=(224, 224),
                                             class_mode=None,
                                             classes=class_subset,
                                             batch_size=1,
                                             shuffle=False,
                                             seed=42)

Found 1933 images belonging to 16 classes.
Found 332 images belonging to 16 classes.
Found 749 images belonging to 16 classes.


In [5]:
class_subset

['P-01',
 'P-04',
 'P-08',
 'P-09',
 'P-1',
 'P-11',
 'P-13',
 'P-16',
 'P-19',
 'P-21',
 'P-22',
 'P-3',
 'P-62',
 'P-69_',
 'P-70',
 'P17']

In [6]:
from tensorflow.keras.models import Sequential


class AlexNet(Model):
    def __init__(self, input_shape: tuple[int, int, int], num_classes: int) -> Model:
        """_summary_

        Args:
            input_shape (tuple[int, int, int]): Input Shape of images  where is (Height, Width, Channel)
            num_classes (int): _description_

        Returns:
            Sequential: Return a models using the AdamNet Architeture for CNN 
        """
        super(AlexNet, self).__init__()

        self.conv1 = Conv2D(96, kernel_size=(11, 11), strides=4, padding='valid',
                            activation='relu', input_shape=input_shape, kernel_initializer='he_normal')
        self.pool1 = MaxPooling2D(pool_size=(3, 3), strides=(
            4, 4), padding='valid', data_format=None)

        self.conv2 = Conv2D(256, kernel_size=(5, 5), strides=1,
                            activation='relu', padding='same', kernel_initializer='he_normal')

        self.pool2 = MaxPooling2D(pool_size=(3, 3), strides=(
            2, 2), padding='valid', data_format=None)

        self.conv3 = Conv2D(384, kernel_size=(3, 3), strides=1, padding='same',
                            activation='relu', kernel_initializer='he_normal')

        self.conv4 = Conv2D(384, kernel_size=(3, 3), strides=1, padding='same',
                            activation='relu', kernel_initializer='he_normal')

        self.conv5 = Conv2D(256, kernel_size=(3, 3), strides=1, padding='same',
                            activation='relu', kernel_initializer='he_normal')

        self.pool3 = MaxPooling2D(pool_size=(3, 3), strides=(
            2, 2), padding='valid', data_format=None)

        self.flatten = Flatten()
        self.fc1 = Dense(4096, activation='relu')
        self.dropout1 = Dropout(0.5)
        self.fc2 = Dense(4096, activation='relu')
        self.dropout2 = Dropout(0.5)
        self.fc3 = Dense(1000, activation='relu')
        self.output_layer = Dense(num_classes, activation='softmax')

   

    def call(self, inputs):
        """
        Define a lógica para o forward pass do modelo.

        Args:
            inputs (tensor): Tensor de entrada.

        Returns:
            tensor: Tensor de saída após passar por todas as camadas.
        """
        x = self.conv1(inputs)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.pool3(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        return self.output_layer(x)

In [7]:
def create_model(input_shape, n_classes, optimizer='rmsprop', fine_tune=0):
    """
    Compiles a model integrated with VGG16 pretrained layers

    input_shape: tuple - the shape of input images (width, height, channels)
    n_classes: int - number of classes for the output layer
    optimizer: string - instantiated optimizer to use for training. Defaults to 'RMSProp'
    fine_tune: int - The number of pre-trained layers to unfreeze.
                If set to 0, all pretrained layers will freeze during training
    """

    # Pretrained convolutional layers are loaded using the Imagenet weights.
    # Include_top is set to False, in order to exclude the model's fully-connected layers.
    conv_base = AlexNet(input_shape, n_classes)


    # Criar um tensor de entrada explícito
    input_tensor = Input(shape=input_shape)
    base_output = conv_base(input_tensor)  # Obter a saída do modelo base



    # Defines how many layers to freeze during training.
    # Layers in the convolutional base are switched from trainable to non-trainable
    # depending on the size of the fine-tuning parameter.
    if fine_tune > 0:
        for layer in conv_base.layers[:-fine_tune]:
            layer.trainable = False
    else:
        for layer in conv_base.layers:
            layer.trainable = False

    # Create a new 'top' of the model (i.e. fully-connected layers).
    # This is 'bootstrapping' a new top_model onto the pretrained layers.
    top_model = conv_base.output
    top_model = Flatten(name="flatten")(top_model)
    top_model = Dense(4096, activation='relu')(top_model)
    top_model = Dense(1072, activation='relu')(top_model)
    top_model = Dropout(0.2)(top_model)
    output_layer = Dense(n_classes, activation='softmax')(top_model)

    # Group the convolutional base and new fully-connected layers into a Model object.
    model = Model(inputs=conv_base.input, outputs=output_layer)

    print(model.summary())

    # Compiles the model for training.
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])

    return model

In [8]:
input_shape = (224, 224, 3)
optim_1 = Adam(learning_rate=0.001)
n_classes= len(class_subset)

n_steps = training.samples // BATCH_SIZE
n_val_steps = validation.samples // BATCH_SIZE
n_epochs = 50

In [9]:


plot_loss_1 = PlotLossesCallback()

# ModelCheckpoint callback - save best weights
tl_checkpoint_1 = ModelCheckpoint(filepath='../../weights/AlexNet_v1.weights.best.keras',
                                  save_best_only=True,
                                  verbose=1)

# EarlyStopping
early_stop = EarlyStopping(monitor='val_loss',
                           patience=10,
                           restore_best_weights=True,
                           mode='min')

In [10]:

# Use a smaller learning rate
optim_2 = Adam(learning_rate=0.0001)

# Just Feature Extraction
vgg_model_ft = create_model(input_shape, n_classes, optim_2, fine_tune=2)

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 alex_net (AlexNet)          (None, 16)                28839928  
                                                                 
 flatten (Flatten)           (None, 16)                0         
                                                                 
 dense_4 (Dense)             (None, 4096)              69632     
                                                                 
 dense_5 (Dense)             (None, 1072)              4391984   
                                                                 
 dropout_2 (Dropout)         (None, 1072)              0         
                                                                 
 dense_6 (Dense)             (None, 16)                17168 

2025-01-19 14:46:00.990948: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-01-19 14:46:01.013825: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-01-19 14:46:01.013982: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:980] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2025-01-19 14:46:01.014897: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

In [11]:
%%time

plot_loss_2 = PlotLossesCallback()


# Retrain model with fine-tuning
vgg_ft_history = vgg_model_ft.fit(training,
                                  batch_size=BATCH_SIZE,
                                  epochs=n_epochs,
                                  validation_data=validation,
                                  steps_per_epoch=n_steps,
                                  validation_steps=n_val_steps,
                                  callbacks=[tl_checkpoint_1, early_stop, plot_loss_2],
                                  verbose=1)

Epoch 1/50


2025-01-19 14:46:03.700043: I tensorflow/stream_executor/cuda/cuda_dnn.cc:384] Loaded cuDNN version 8400


 1/30 [>.............................] - ETA: 1:08 - loss: 2.7715 - accuracy: 0.0625

2025-01-19 14:46:04.564705: I tensorflow/stream_executor/cuda/cuda_blas.cc:1614] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


30/30 [==============================] - ETA: 0s - loss: 2.7151 - accuracy: 0.1182

KeyboardInterrupt: 

In [ ]:

# Loading predictions from last article's model
from sklearn.metrics import accuracy_score

true_classes = testing.classes
# Generate predictions
vgg_model_ft.load_weights('../../weights/AlexNet_v1.weights.best.keras') # initialize the best trained weights

vgg_preds_ft = vgg_model_ft.predict(testing)
vgg_pred_classes_ft = n.argmax(vgg_preds_ft, axis=1)
vgg_acc_ft = accuracy_score(true_classes, vgg_pred_classes_ft)
print("VGG16 Model Accuracy with Fine-Tuning: {:.2f}%".format(vgg_acc_ft * 100))


510/749 [===================>..........] - ETA: 0s

In [ ]:
import seaborn as sns
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt



# Get the names of the ten classes
class_names = testing.class_indices.keys()

def plot_heatmap(y_true, y_pred, class_names, ax, title):
    cm = confusion_matrix(y_true, y_pred)
    sns.heatmap(
        cm,
        annot=True,
        square=True,
        xticklabels=class_names,
        yticklabels=class_names,
        fmt='d',
        cmap=plt.cm.Blues,
        cbar=False,
        ax=ax
    )
    ax.set_title(title, fontsize=16)
    ax.set_xticklabels(ax.get_xticklabels(), rotation=45, ha="right")
    ax.set_ylabel('True Label', fontsize=12)
    ax.set_xlabel('Predicted Label', fontsize=12)

fig, ax = plt.subplots(figsize=(20, 10))

plot_heatmap(true_classes, vgg_pred_classes_ft, class_names, ax, title="Transfer Learning (VGG16) with Fine-Tuning")


fig.tight_layout()
fig.subplots_adjust(top=1.25)
plt.show()
